In [51]:
import pandas as pd
import numpy as np

#### Bringing in all the Player Data that has Yardage data to be aggregated by team.

In [74]:
pstats = pd.read_csv('./data/player_stats_2005.csv', index_col = "date", parse_dates = True)

In [75]:
pstats.sort_index(inplace=True)
pstats.drop(columns = 'Unnamed: 0', inplace=True)

In [76]:
pstats = pstats[pstats['year'] >= 2010]

In [77]:
pstats.shape

(181675, 45)

In [78]:
pstats = pstats[pstats['game_number'] <= 16]

In [79]:
pstats.shape

(174889, 45)

In [80]:
pstats['home_team'] = np.where(pstats['game_location'] == 'H', pstats['team'], pstats['opponent'])

In [81]:
pstats.reset_index(inplace=True)

In [82]:
pstats.columns

Index(['date', 'player_id', 'year', 'game_number', 'age', 'team',
       'game_location', 'opponent', 'game_won', 'player_team_score',
       'opponent_score', 'passing_attempts', 'passing_completions',
       'passing_yards', 'passing_rating', 'passing_touchdowns',
       'passing_interceptions', 'passing_sacks', 'passing_sacks_yards_lost',
       'rushing_attempts', 'rushing_yards', 'rushing_touchdowns',
       'receiving_targets', 'receiving_receptions', 'receiving_yards',
       'receiving_touchdowns', 'kick_return_attempts', 'kick_return_yards',
       'kick_return_touchdowns', 'punt_return_attempts', 'punt_return_yards',
       'punt_return_touchdowns', 'defense_sacks', 'defense_tackles',
       'defense_tackle_assists', 'defense_interceptions',
       'defense_interception_yards', 'defense_interception_touchdowns',
       'defense_safeties', 'point_after_attemps', 'point_after_makes',
       'field_goal_attempts', 'field_goal_makes', 'punting_attempts',
       'punting_yards', '

In [83]:
pstats = pstats[['date','player_id', 'game_number', 'team', 'home_team', 'opponent', 'game_won', 'player_team_score', 'opponent_score',
    'passing_yards', 'rushing_yards', 'receiving_yards']]

In [84]:
pstats.head(1)

,date,player_id,game_number,team,home_team,opponent,game_won,player_team_score,opponent_score,passing_yards,rushing_yards,receiving_yards
0,2010-09-09,3778,1,NOR,NOR,MIN,True,14,9,0,0,0


In [85]:
pstats['date'] = pstats['date'].astype(str)

In [86]:
nfl_yards_agg = pd.DataFrame(pstats.groupby(['date','game_number', 'team']).sum()[['passing_yards','rushing_yards']])
nfl_yards_agg.reset_index(inplace=True)

In [87]:
nfl_yards_agg[nfl_yards_agg['team']== 'STL']

,date,game_number,team,passing_yards,rushing_yards
24,2010-09-12,1,STL,253,85
58,2010-09-19,2,STL,167,75
90,2010-09-26,3,STL,235,133
119,2010-10-03,4,STL,289,88
147,2010-10-10,5,STL,215,128
...,...,...,...,...,...
2939,2015-12-06,12,STL,146,66
2970,2015-12-13,13,STL,124,203
2976,2015-12-17,14,STL,234,98
3035,2015-12-27,15,STL,103,102


In [92]:
nfl_yards_agg['team'].nunique()

32

- mapping team id's

In [89]:
def desc_to_id(id):

    id_dict = {
       
        'GNB': "GB",
        'NOR': 'NO',
        'TAM': 'TB',
        'KAN': 'KC',
        'NWE': 'NE',
        'SFO': 'SF',
        'ARI': 'ARI',
        'ATL': 'ATL',
        'BAL': 'BAL',
        'BUF': 'BUF',
        'CAR': 'CAR',
        'CHI': 'CHI',
        'CIN': 'CIN',
        'CLE': 'CLE',
        'DAL': 'DAL',
        'DEN': 'DEN',
        'DET': 'DET',
        'HOU': 'HOU',
        'IND': 'IND',
        'JAX': 'JAX',
        'LAC': 'LAC',
        'LAR': 'LAR',
        'MIA': 'MIA',        
        'MIN': 'MIN',
        'NYG': 'NYG',
        'NYJ': 'NYJ',
        'OAK': 'OAK',
        'PHI': 'PHI',
        'PIT': 'PIT',
        'SDG': 'LAC',
        'SEA': 'SEA',
        'STL': 'LAR',
        'TEN': 'TEN',
        'WAS': 'WAS',
            }
    
    try:
        return id_dict[id]
    except:
        return 'CHECK!'

In [90]:
nfl_yards_agg['team'] = nfl_yards_agg['team'].apply(desc_to_id)

In [91]:
nfl_yards_agg['team'].nunique()

32

- creating unique team id with date

In [97]:
nfl_yards_agg['id'] = nfl_yards_agg['team'] + nfl_yards_agg['date']

In [98]:
nfl_yards_agg.head()

,date,game_number,team,passing_yards,rushing_yards,id
0,2010-09-09,1,MIN,171,91,MIN2010-09-09
1,2010-09-09,1,NO,237,79,NO2010-09-09
2,2010-09-12,1,ARI,297,112,ARI2010-09-12
3,2010-09-12,1,ATL,252,58,ATL2010-09-12
4,2010-09-12,1,BUF,139,50,BUF2010-09-12


#### Bringing in Scores data

In [107]:
nfl_scores = pd.read_csv('./data/nfl_scores_agg.csv', index_col=0)

In [108]:
nfl_scores.head()

,schedule_date,schedule_season,schedule_week,team,score,opponent_score,opponent_team,team_favorite_id,spread_favorite,over_under_line,margin,home_team,id
0,2010-09-09,2010,1,MIN,9.0,14.0,NO,NO,-5.0,49.5,-5.0,NO,MIN2010-09-09
0,2010-09-09,2010,1,NO,14.0,9.0,MIN,NO,-5.0,49.5,5.0,NO,NO2010-09-09
10,2010-09-12,2010,1,ARI,17.0,13.0,LAR,ARI,-3.0,39.5,4.0,LAR,ARI2010-09-12
8,2010-09-12,2010,1,ATL,9.0,15.0,PIT,ATL,-1.5,39.5,-6.0,PIT,ATL2010-09-12
1,2010-09-12,2010,1,BUF,10.0,15.0,MIA,MIA,-3.0,39.0,-5.0,BUF,BUF2010-09-12


In [109]:
nfl_agg_agg = pd.merge(nfl_scores, nfl_yards_agg, on = 'id')

In [112]:
nfl_agg_agg.drop(columns = ['team_y', 'date'], inplace=True)

In [113]:
nfl_agg_agg.tail()

,schedule_date,schedule_season,schedule_week,team_x,score,opponent_score,opponent_team,team_favorite_id,spread_favorite,over_under_line,margin,home_team,id,game_number,passing_yards,rushing_yards
3933,2017-11-26,2017,12,TEN,20.0,16.0,IND,TEN,-3.5,44.0,4.0,IND,TEN2017-11-26,11,184,92
3934,2017-11-27,2017,12,BAL,23.0,16.0,HOU,BAL,-7.0,38.0,7.0,BAL,BAL2017-11-27,11,163,139
3935,2017-11-27,2017,12,HOU,16.0,23.0,BAL,BAL,-7.0,38.0,-7.0,BAL,HOU2017-11-27,11,252,66
3936,2017-11-30,2017,13,DAL,38.0,14.0,WAS,WAS,-2.0,47.0,24.0,DAL,DAL2017-11-30,12,102,182
3937,2017-11-30,2017,13,WAS,14.0,38.0,DAL,WAS,-2.0,47.0,-24.0,DAL,WAS2017-11-30,12,0,50


In [116]:
nfl_agg_agg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3938 entries, 0 to 3937
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   schedule_date     3938 non-null   object 
 1   schedule_season   3938 non-null   int64  
 2   schedule_week     3938 non-null   int64  
 3   team_x            3938 non-null   object 
 4   score             3938 non-null   float64
 5   opponent_score    3938 non-null   float64
 6   opponent_team     3938 non-null   object 
 7   team_favorite_id  3938 non-null   object 
 8   spread_favorite   3938 non-null   float64
 9   over_under_line   3938 non-null   float64
 10  margin            3938 non-null   float64
 11  home_team         3938 non-null   object 
 12  id                3938 non-null   object 
 13  game_number       3938 non-null   int64  
 14  passing_yards     3938 non-null   int64  
 15  rushing_yards     3938 non-null   int64  
dtypes: float64(5), int64(5), object(6)
memory 

In [115]:
nfl_agg_agg[nfl_agg_agg['team_x']== 'NYJ']

,schedule_date,schedule_season,schedule_week,team_x,score,opponent_score,opponent_team,team_favorite_id,spread_favorite,over_under_line,margin,home_team,id,game_number,passing_yards,rushing_yards
31,2010-09-13,2010,1,NYJ,9.0,10.0,BAL,NYJ,-1.0,36.5,-1.0,NYJ,NYJ2010-09-13,1,74,116
54,2010-09-19,2010,2,NYJ,28.0,14.0,NE,NE,-3.0,39.5,14.0,NYJ,NYJ2010-09-19,2,220,136
85,2010-09-26,2010,3,NYJ,31.0,23.0,MIA,MIA,-2.5,36.0,8.0,MIA,NYJ2010-09-26,3,256,146
114,2010-10-03,2010,4,NYJ,38.0,14.0,BUF,NYJ,-6.0,37.0,24.0,BUF,NYJ2010-10-03,4,171,273
151,2010-10-11,2010,5,NYJ,29.0,20.0,MIN,NYJ,-4.5,40.0,9.0,NYJ,NYJ2010-10-11,5,191,155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3788,2017-10-22,2017,7,NYJ,28.0,31.0,MIA,MIA,-3.0,39.5,-3.0,MIA,NYJ2017-10-22,7,209,92
3812,2017-10-29,2017,8,NYJ,20.0,25.0,ATL,ATL,-4.5,46.0,-5.0,NYJ,NYJ2017-10-29,8,257,43
3823,2017-11-02,2017,9,NYJ,34.0,21.0,BUF,BUF,-3.0,43.0,13.0,NYJ,NYJ2017-11-02,9,140,194
3868,2017-11-12,2017,10,NYJ,10.0,15.0,TB,TB,-1.0,44.5,-5.0,TB,NYJ2017-11-12,10,262,56


In [117]:
nfl_agg_agg.to_csv('./data/nfl_merged_all.csv')